In [ ]:
!apt-get update && apt-get install ffmpeg tmux vim -y
!pip3 install moviepy openai python-dotenv pydub pytubefix openai-whisper llama-index llama-index-llms-openai llama-index-llms-ollama llama-index-embeddings-ollama 
!pip3 install flash-attn --no-build-isolation
!curl -fsSL https://ollama.com/install.sh | sh

# !chmod +x /usr/bin/ollama
# !useradd -r -s /bin/false -m -d /usr/share/ollama ollama
# !pip3 install openpyxl sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv dspy-ai graspologic


In [ ]:
import os, pickle, gc, subprocess
from dotenv import load_dotenv
from video_transcription import split_audio_into_chunks, video_to_audio, transcribe_audio_chunks, get_transcription_model, download_video
from llama_index.core import Document
# import torch

load_dotenv("/workspace/repos/agentic-ai/.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

video_path="/workspace/data/video1512218125.mp4"
# video_path="https://www.youtube.com/watch?v=Z07Ewop7rQA"
audio_output_path="/workspace/data/video_audio.mp3"
transcribe_output_dir="/workspace/data"
file_save_path="/workspace/data/transcription.pkl"

In [ ]:
if not os.path.exists(file_save_path):
    print("Loading transcription model...")
    transcribe_model = get_transcription_model(open_source_model=True)

    print("Processing video...")
    if 'youtube' in video_path:
        print("   Downloading video from youtube...")
        download_video(video_url=video_path, audio_output_path=audio_output_path)
    else:
        video_to_audio(video_path=video_path, audio_output_path=audio_output_path)

    print("Splitting audio into chunks...")
    split_audio_into_chunks(audio_output_path=audio_output_path, transcribe_output_dir=transcribe_output_dir, max_chunk_size_mb=24)

    print("Transcribing audio chunks...")
    documents = transcribe_audio_chunks(model=transcribe_model, chunk_dir="/workspace/data", file_save_path=file_save_path)

    del transcribe_model
    # torch.cuda.empty_cache()
    gc.collect()
else:
    with open(file_save_path, 'rb') as f:
        documents = pickle.load(f)
    
if isinstance(documents[0], dict):
    documents = [Document(text=chunk["text"]) for chunk in documents]

full_doc = " ".join([doc.text for doc in documents])


In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI

# model_name, ctx_len = "gpt-4o", 128000
# model_name, ctx_len = "llama3.1:latest", 128000
model_name, ctx_len = "hermes3:8b", 128000

if model_name == "gpt-4o":
    llm = LOpenAI(model=model_name, max_tokens=4000)
else:
    try: 
        print("Pulling Ollama model...")
        sub_out = subprocess.run(['ollama', 'pull', model_name], capture_output=True, text=True)
    except Exception as e: 
        print(f"Error pulling model: Is the Ollama server running?\n{e}")
    
    addtion_kwargs = {"max_new_tokens": 4000}
    # system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any sections, rules, acts or laws (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) from context that support the answer. Be detailed in your response."
    llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=False, 
                request_timeout=1000.0, additional_kwargs=addtion_kwargs) # system_prompt=system_prompt
    print(llm.metadata)

Settings.llm = llm

## Tree summarizer

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(llm=llm, verbose=True)
# prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."
prompt_summary = f"""You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it the memo has the following items:
1. Extract all the names of new hires and their position, and/or new advisors mentioned in the transcription.
2. Create a section to mention the personnel new to AlphaTrAI.
3. Include other highlights and progress made by AlphaTrAI.
4. Ensure the memo and ensure it is factual, optimistic, and any values mention come directly from the text. 

The transcription is as follows:\n{full_doc}"""

response = await summarizer.aget_response(prompt_summary, [doc.text for doc in documents])

In [ ]:
print(response)

## LLM direct summarization

In [ ]:
prompt_summary = f"""You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it the memo has the following items:
1. Extract all the names of new hires and their position, and/or new advisors mentioned in the transcription.
2. Include other highlights and progress made by AlphaTrAI.
3. Ensure the memo is professional, fluid, factual, and optimistic. 

The transcription is as follows:\n{full_doc}"""

response = llm.complete(prompt_summary, max_tokens=5000)

In [ ]:
print(response.text)

## Agentic Summary

In [ ]:
!pip3 install llama-index-embeddings-huggingface llama-index-vector-stores-neo4jvector llama-index-graph-stores-neo4j
!apt install dialog apt-utils -y (done above)
!wget -O - https://debian.neo4j.com/neotechnology.gpg.key | gpg --dearmor -o /etc/apt/keyrings/neotechnology.gpg
!echo 'deb [signed-by=/etc/apt/keyrings/neotechnology.gpg] https://debian.neo4j.com stable latest' | tee -a /etc/apt/sources.list.d/neo4j.list
!apt list -a neo4j
!add-apt-repository universe -y
!apt install neo4j=1:5.22.0 -y
!echo "neo4j-enterprise neo4j/question select I ACCEPT" | debconf-set-selections
!echo "neo4j-enterprise neo4j/license note" | debconf-set-selections
!apt install openjdk-17-jre -y
!cd /var/lib/neo4j/plugins/ && wget https://github.com/neo4j/apoc/releases/download/5.22.0/apoc-5.22.0-core.jar

In [ ]:
set_neo4j_password('bewaretheneo')
add_lines_to_conf()


In [ ]:
from dotenv import load_dotenv
load_dotenv()

from llama_index.core.agent import ReActAgent
# from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from rag_utils import create_neo4j_graph_store, create_neo4j_graphrag, neo4j_query, set_neo4j_password, add_lines_to_conf
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import nest_asyncio
nest_asyncio.apply()



In [ ]:

llm.is_function_calling_model = True

embed_model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
print("loading embed model...")
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

Settings.embed_model = embed_model
Settings.chunk_size = 300
Settings.chunk_overlap = 50

In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

entities = Literal["PEOPLE", 
                   "PLACE"
]

relations = Literal[
    "ROLE",
    "COMPANY"
]

validation_schema = {
    "People": ["ROLE"],
    "Place": ["COMPANY"],
}


In [ ]:
Settings.chunk_size = 300
Settings.chunk_overlap = 50

kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=validation_schema,
    strict=True,  # if false, will allow triples outside of the schema
    num_workers=4,
    max_triplets_per_chunk=10,
)

graph_store = create_neo4j_graph_store(neo_url="bolt://localhost:7687", 
                                       password=os.getenv("NEO4J_PWD"), 
                                       config={"connection_timeout": 240, "connection_acquisition_timeout": 240, "max_connection_pool_size": 1000})
neo4j_query(graph_store, query="""MATCH (n) DETACH DELETE n""")


graph_index = create_neo4j_graphrag(documents, llm, embed_model, kg_extractor, graph_store)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = QueryEngineTool(
            query_engine=graph_index,
            metadata=ToolMetadata(
                name="graph_tool",
                description=(
                    "Useful for finding people names and roles, and the company they work for."
                ),
            ),
        ),


In [ ]:
!pip3 install llama-agents

In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import ReActAgentWorker, ReActAgent



# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=[query_engine_tools],#, adding_tool],
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in [query_engine_tools]#, adding_tool]
]


# define Agent and agent service
# worker1 = FunctionCallingAgentWorker.from_tools(
worker1 = ReActAgentWorker.from_tools(
    meta_tools,
    llm=llm,
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Summarize a transcription as a memo for investors and stakeholders.",
    service_name="summarize_transcription",
)